In [3]:
!pip uninstall -y -ensorflow 


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: -e


In [ ]:
!pip install tensorflow-gpu

In [1]:
!pip uninstall -y efficientnet

Found existing installation: efficientnet 1.1.1
Uninstalling efficientnet-1.1.1:
  Successfully uninstalled efficientnet-1.1.1


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15039468992037813273]

In [4]:
import tensorflow as tf
import keras
import tensorflow_hub as hub
import numpy as np
from keras.preprocessing import image
from glob import glob
import cv2
from sklearn.model_selection import train_test_split
import os
import random
import pandas as pd
import keras.layers as layers

In [5]:
dir_path="../Fooddata/Data/food-101/food-101/images/*"

# データフレーム作成

In [6]:
all_path=glob("../Fooddata/Data/food-101/food-101/images/*/*")

In [7]:
train_path,test_path=train_test_split(all_path,train_size=0.7,test_size=0.3)

In [8]:
train_label=[i.split(os.path.sep)[-2] for i in train_path]

In [9]:
len(set(train_label))

101

In [10]:
test_label=[i.split(os.path.sep)[-2] for i in test_path]

In [11]:
len(set(test_label))

101

In [12]:
train_df=pd.DataFrame([train_path,train_label]).T       # , index=['filename'，'class']
train_df.columns=["filename","class"]

In [13]:
test_df=pd.DataFrame([test_path,test_label]).T       # , index=['filename'，'class']
test_df.columns=["filename","class"]

In [14]:
test_df

,filename,class
0,../Fooddata/Data/food-101/food-101/images/frie...,fried_rice
1,../Fooddata/Data/food-101/food-101/images/edam...,edamame
2,../Fooddata/Data/food-101/food-101/images/muss...,mussels
3,../Fooddata/Data/food-101/food-101/images/chic...,chicken_quesadilla
4,../Fooddata/Data/food-101/food-101/images/spri...,spring_rolls
...,...,...
30295,../Fooddata/Data/food-101/food-101/images/pizz...,pizza
30296,../Fooddata/Data/food-101/food-101/images/choc...,chocolate_mousse
30297,../Fooddata/Data/food-101/food-101/images/carr...,carrot_cake
30298,../Fooddata/Data/food-101/food-101/images/nach...,nachos


In [15]:
train_df

,filename,class
0,../Fooddata/Data/food-101/food-101/images/oyst...,oysters
1,../Fooddata/Data/food-101/food-101/images/eggs...,eggs_benedict
2,../Fooddata/Data/food-101/food-101/images/frie...,fried_rice
3,../Fooddata/Data/food-101/food-101/images/beet...,beet_salad
4,../Fooddata/Data/food-101/food-101/images/riso...,risotto
...,...,...
70695,../Fooddata/Data/food-101/food-101/images/baby...,baby_back_ribs
70696,../Fooddata/Data/food-101/food-101/images/fren...,french_fries
70697,../Fooddata/Data/food-101/food-101/images/donu...,donuts
70698,../Fooddata/Data/food-101/food-101/images/spag...,spaghetti_carbonara


# Model作成

In [16]:
#Input=keras.layers.Input((224,224,3))

In [17]:
EffBlock=tf.keras.applications.EfficientNetB2( include_top=False,weights='imagenet', input_shape=(224,224,3))
    
    
"""
    include_top=False,
    weights="imagenet",
    input_tensor=Input,
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax")
"""

'\n    include_top=False,\n    weights="imagenet",\n    input_tensor=Input,\n    input_shape=(224,224,3),\n    pooling=None,\n    classes=1000,\n    classifier_activation="softmax")\n'

In [18]:
EffNet=keras.Sequential([
    EffBlock,
    layers.MaxPooling2D(7),
    layers.Flatten(),
    layers.Dense(512,input_shape=(1408,),activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(128,input_shape=(512,),activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(101,input_shape=(128,),activation="softmax")
    ])

In [19]:
EffBlock.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [20]:
EffNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb2 (Functional)  (None, 7, 7, 1408)        7768569   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 1408)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1408)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               721408    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

# DataLoader作成

In [21]:
np.array(train_df.iloc[1])[1]

'eggs_benedict'

In [22]:
for i in []:
    print("DFGHJKL")

In [23]:
class DataLoader:
    
    class DataSet:
        def __init__(self,paths):
            self.data_paths=paths
            self.label_dict={}
            parent=glob("../Fooddata/Data/food-101/food-101/images/*")
            for i,v in enumerate(parent):
                self.label_dict[v.split(os.path.sep)[-1]]=np.identity(len(parent))[i]

        def __getitem__(self,item):
            return (self.data_paths[item],self.label_dict[self.data_paths[item].split(os.path.sep)[-2]])
        
    def __init__(self,DataFrame):
        self.data=self.DataSet(DataFrame)
        self.label_dict=self.data.label_dict
        self.reset()
        
    def reset(self):
        self.yeild_img=[]
        self.yeild_label=[]
    
    def flow(self,batch_num,transforms=[]):
        self.reset()
        #i_path:::str,i_label:::narray
        for i_path,i_label in self.data:
            tem_img=cv2.imread(i_path)
            for func in transforms:
                tem_img=func(tem_img)
            self.yeild_img.append(tem_img.copy())
            self.yeild_label.append(i_label.copy())
            if len(self.yeild_img)==batch_num:
                yield (np.array(self.yeild_img),np.array(self.yeild_label))
                self.reset()
        else:
            if(len(yeild_label)>0):
                yield (np.array(self.yeild_img),np.array(self.yeild_label))
            

In [24]:
def RandomCrop(img):
    height, width, channels = img.shape
    cropsize=np.min([height, width])
    pos_axis=np.argmin([height, width])
    result=[]
    
    if pos_axis==0:
        pos_start=random.randint(0,width-height+1)
        result=img[:,pos_start:pos_start+height,:]
    else:
        pos_start=random.randint(0,height-width+1)
        result=img[pos_start:pos_start+width,:,:]
    
    return cv2.resize(result, (224,224))

In [25]:
def H_flip(img):
    if(random.uniform(0,1)>0.5):
        return img[::-1,:,:]
    return img

In [26]:
def W_flip(img):
    if(random.uniform(0,1)>0.5):
        return img[:,::-1,:]
    return img
def Scaled(img):
    return img*(1.0/255)

In [27]:
BATCH_NUM=100
EPOCH=1000
TrainLoader=DataLoader(train_path).flow(BATCH_NUM,[RandomCrop,H_flip,W_flip,Scaled])
TestLoader=DataLoader(test_path).flow(BATCH_NUM,[RandomCrop])

In [28]:
EffNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
EffNet.fit_generator( generator=TrainLoader,
    epochs=EPOCH,
    steps_per_epoch=int(np.ceil(len(train_path) / BATCH_NUM)),
    verbose=1,
    validation_data=TestLoader)

/home/is/shota-nak/.local/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
477/707 [===================>..........] - ETA: 21:07 - loss: 4.5927 - accuracy: 0.0203

KeyboardInterrupt: 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
Data_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255)

In [ ]:
test_DataSet=Data_gen.flow_from_dataframe(test_df,target_size=(224,224),batch_size=10,shuffle=False)
train_DataSet=Data_gen.flow_from_dataframe(train_df,target_size=(224,224),batch_size=10,shuffle=True)

#flow_from_dataframe(test_df, target_size=(224， 224),batch_size=32,shuffle=False)

In [ ]:
EffNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
EffNet.fit_generator( generator=TrainLoader,
    epochs=EPOCH,
    steps_per_epoch=int(np.ceil(len(train_path) / 10)),
    verbose=1,
    validation_data=TestLoader)